<a href="https://colab.research.google.com/github/EEG-Easy/EEG-Machinelearning/blob/master/2020-2Project/2_%EB%94%A5%EB%9F%AC%EB%8B%9D/P300RobustScaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My Drive/eeg

/content/gdrive/My Drive/eeg


In [0]:
EEG_df = pd.read_csv('P300.csv')

In [0]:
EEG_df.pop('subject')
EEG_df.pop('trial')
EEG_df.columns

Index(['condition', 'group', 'F1_P300', 'FC3_P300', 'FC1_P300', 'AFz_P300',
       'Fz_P300', 'F2_P300', 'FCz_P300'],
      dtype='object')

In [0]:
condition=np.array(EEG_df['condition'])
group=np.array(EEG_df['group'])

EEG_scaling = EEG_df.drop(['condition', 'group'], axis=1)

In [0]:
scaler = RobustScaler()

In [0]:
scaler.fit(EEG_scaling) 
EEG_scaled = scaler.transform(EEG_scaling)
EEG_scaled_df = pd.DataFrame(EEG_scaled)

In [0]:
EEG_scaled_df.columns = ['F1', 'FC3', 'FC1', 'AFz', 'Fz', 'F2', 'FCz']
EEG_scaled_df['condition'] = condition
EEG_scaled_df['group'] = group
EEG_scaled_df.head(5)

,F1,FC3,FC1,AFz,Fz,F2,FCz,condition,group
0,-0.193157,-0.093217,-0.139332,-0.014351,-0.482007,-0.612296,-0.432072,1,0
1,0.887894,1.593248,1.007132,0.268088,0.678681,1.014418,1.029642,1,0
2,0.254117,0.390429,0.204192,-0.012284,0.633721,0.676659,0.360667,1,0
3,0.415138,0.628633,1.080357,0.549231,0.658604,0.430925,0.769865,1,0
4,-0.608327,-0.192824,-0.775245,-0.364559,-0.693839,-0.516647,-0.817077,1,0


In [0]:
EEG_scaled_df.to_csv('/content/gdrive/My Drive/eeg/RobustP300.csv',sep=',', na_rep='NaN')

In [0]:
LABEL_COLUMN = 'group'
LABELS = [0, 1]

CSV_COLUMNS = ['condition','group', 'F1', 'FC3', 'FC1', 'AFz', 'Fz', 'F2', 'FCz']

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

temp_dataset = get_dataset('RobustP300.csv', select_columns=CSV_COLUMNS)

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['F1', 'FC3', 'FC1', 'AFz', 'Fz', 'F2', 'FCz']

packed_data = temp_dataset.map(
    PackNumericFeatures(NUMERIC_FEATURES))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'


In [0]:
import pandas as pd
desc = EEG_scaled_df[NUMERIC_FEATURES].describe()
desc

,F1,FC3,FC1,AFz,Fz,F2,FCz
count,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000,23201.000000
mean,0.169534,0.147238,0.125680,0.257439,0.161898,0.181233,0.115794
std,1.137716,1.068238,1.004288,1.523621,1.147731,1.186636,0.957781
min,-8.638409,-7.128760,-6.194677,-10.138734,-7.300168,-7.736735,-6.063609
25%,-0.450238,-0.453220,-0.460870,-0.440929,-0.455397,-0.448035,-0.462869
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.549762,0.546780,0.539130,0.559071,0.544603,0.551965,0.537131
max,17.795594,20.475962,24.351948,25.661585,17.733415,16.691511,13.711209


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(7,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7efc5a8ddd08>, mean=array([0.16953369, 0.14723849, 0.1256803 , 0.25743903, 0.16189793,
       0.18123256, 0.11579391]), std=array([1.13771572, 1.06823785, 1.00428802, 1.52362056, 1.14773099,
       1.18663606, 0.95778078])))

In [0]:
example_batch, labels_batch = next(iter(packed_data)) 
example_batch['numeric']

<tf.Tensor: shape=(5, 7), dtype=float32, numpy=
array([[-0.73736995, -1.0586225 , -0.76923674, -0.7760385 , -0.79601073,
        -0.9157964 , -1.6266463 ],
       [-0.00393089, -0.19127612, -0.29434994,  0.05472529, -0.11946619,
         0.14347596, -0.2100413 ],
       [-0.31780148,  0.32996264,  0.14018953,  0.09033884, -0.37287387,
        -0.38955414,  0.38241202],
       [-0.19593135,  0.448088  ,  0.13790326,  0.05126667, -0.14704439,
         0.10892434,  0.22845288],
       [-0.531504  , -0.26355326, -1.2163053 , -0.4796184 , -0.61335254,
        -0.68536085, -0.9029014 ]], dtype=float32)>

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.79712677, -1.1288319 , -0.89109594, -0.6783037 , -0.83461076,
        -0.92448646, -1.8192475 ],
       [-0.15246741, -0.31689066, -0.4182368 , -0.13304737, -0.24514815,
        -0.03181817, -0.3401981 ],
       [-0.42834526,  0.17105192,  0.01444728, -0.10967309, -0.4659383 ,
        -0.48101243,  0.2783707 ],
       [-0.32122704,  0.28163156,  0.01217077, -0.13531739, -0.2691766 ,
        -0.06093546,  0.11762501],
       [-0.61618   , -0.3845508 , -1.3362557 , -0.48375392, -0.67546356,
        -0.73029417, -1.0635997 ]], dtype=float32)

In [0]:
CATEGORIES = {'condition': [1, 2, 3]}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 0. 1.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns) #+numeric_columns

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer, # 256,256 / 128,256
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  #tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [0]:
train_data = packed_data.shuffle(20000)
test_data = packed_data.shuffle(1000)

In [0]:
history=model.fit(train_data, epochs=1)

4641/4641 [==============================] - 25s 5ms/step - loss: 0.6708 - accuracy: 0.6017


In [0]:
model.summary()
model.save('model1.h5')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_2 (DenseFeatu multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1408      
_________________________________________________________________
dense_1 (Dense)              multiple                  33024     
_________________________________________________________________
dense_2 (Dense)              multiple                  257       
Total params: 34,689
Trainable params: 34,689
Non-trainable params: 0
_________________________________________________________________
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-1d7412db98ae>", line 2, in <mo

ValueError: ignored

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 19s 4ms/step - loss: 0.5333 - acc: 0.7847


Test Loss 0.5332511613264577, Test Accuracy 0.7847075462341309


In [0]:
model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

Epoch 1/300
4641/4641 [==============================] - 23s 5ms/step - loss: 0.5100 - acc: 0.7814
Epoch 2/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4426 - acc: 0.7880
Epoch 3/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4276 - acc: 0.7912
Epoch 4/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4440 - acc: 0.7902
Epoch 5/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4312 - acc: 0.7913
Epoch 6/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4291 - acc: 0.7938
Epoch 7/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4102 - acc: 0.7963
Epoch 8/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4287 - acc: 0.7943
Epoch 9/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.4074 - acc: 0.8005
Epoch 10/300
4641/4641 [==============================] - 12s 2ms/step - loss: 0.4173 - acc: 0.7985
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 19s 4ms/step - loss: 0.3293 - acc: 0.8490


Test Loss 0.32933354961227723, Test Accuracy 0.8490151166915894


In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

Epoch 1/300
4641/4641 [==============================] - 24s 5ms/step - loss: 0.3421 - acc: 0.8446
Epoch 2/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3258 - acc: 0.8501
Epoch 3/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3170 - acc: 0.8491
Epoch 4/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3408 - acc: 0.8503
Epoch 5/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2976 - acc: 0.8562
Epoch 6/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3238 - acc: 0.8515
Epoch 7/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3108 - acc: 0.8550
Epoch 8/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3124 - acc: 0.8550
Epoch 9/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3097 - acc: 0.8570
Epoch 10/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3248 - acc: 0.8549
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 18s 4ms/step - loss: 0.2420 - acc: 0.8822


Test Loss 0.2420369508353826, Test Accuracy 0.8822464346885681


In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

Epoch 1/200
4641/4641 [==============================] - 24s 5ms/step - loss: 0.2970 - acc: 0.8682
Epoch 2/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2638 - acc: 0.8781
Epoch 3/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.3210 - acc: 0.8683
Epoch 4/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2802 - acc: 0.8725
Epoch 5/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2595 - acc: 0.8754
Epoch 6/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2999 - acc: 0.8714
Epoch 7/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2579 - acc: 0.8804
Epoch 8/200
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2579 - acc: 0.8748
Epoch 9/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2795 - acc: 0.8770
Epoch 10/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2613 - acc: 0.8800
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 18s 4ms/step - loss: 0.2648 - acc: 0.8828


Test Loss 0.26475580574369434, Test Accuracy 0.8828498721122742


In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

Epoch 1/200
4641/4641 [==============================] - 24s 5ms/step - loss: 0.2641 - acc: 0.8803
Epoch 2/200
4641/4641 [==============================] - 12s 2ms/step - loss: 0.2480 - acc: 0.8835
Epoch 3/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2653 - acc: 0.8791
Epoch 4/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2718 - acc: 0.8797
Epoch 5/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2381 - acc: 0.8847
Epoch 6/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2635 - acc: 0.8817
Epoch 7/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2317 - acc: 0.8882
Epoch 8/200
4641/4641 [==============================] - 12s 3ms/step - loss: 0.2489 - acc: 0.8846
Epoch 9/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2609 - acc: 0.8851
Epoch 10/200
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2336 - acc: 0.8873
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 19s 4ms/step - loss: 0.2402 - acc: 0.8894


Test Loss 0.24016748056134873, Test Accuracy 0.8893582224845886


In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

Epoch 1/300
4641/4641 [==============================] - 25s 5ms/step - loss: 0.2451 - acc: 0.8882
Epoch 2/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2454 - acc: 0.8888
Epoch 3/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2547 - acc: 0.8882
Epoch 4/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2370 - acc: 0.8908
Epoch 5/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2754 - acc: 0.8878
Epoch 6/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2352 - acc: 0.8917
Epoch 7/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2536 - acc: 0.8887
Epoch 8/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2279 - acc: 0.8908
Epoch 9/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2286 - acc: 0.8939
Epoch 10/300
4641/4641 [==============================] - 11s 2ms/step - loss: 0.2400 - acc: 0.8897
Epoch 11/

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

4641/4641 [==============================] - 18s 4ms/step - loss: 0.2363 - acc: 0.8928


Test Loss 0.23625113713507082, Test Accuracy 0.8928494453430176


In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

In [0]:
train_data = packed_data.shuffle(15000)
test_data = packed_data.shuffle(1000)

In [0]:
model.fit(train_data, epochs=300)

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))